In [1]:
# Import necessary libraries
%run ./preprocessing.ipynb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb 


Missing values in the dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 1987936 entries, 0 to 1987942
Data columns (total 27 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Order ID              int64  
 1   Account ID            int64  
 2   Market Key            object 
 3   Security ID           int64  
 4   Order Type            object 
 5   Order Time            object 
 6   Order Via             object 
 7   Is Completed          int64  
 8   Is Canceled           int64  
 9   Expire Date           object 
 10  Execution Status      object 
 11  quantity              int64  
 12  Price                 float64
 13  Sector Name           object 
 14  Executed Quantity     int64  
 15  Quantity              int64  
 16  Client ID             int64  
 17  Gender                object 
 18  Risk Rate             object 
 19  Company Name          object 
 20  Is Closed             int64  
 21  Is Dormant            float64
 22  Is Profile Susp

C:\Users\Mohamed\AppData\Local\Temp\ipykernel_21144\3983029155.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=columns_to_remove, inplace=True)
C:\Users\Mohamed\AppData\Local\Temp\ipykernel_21144\3983029155.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(columns=columns_to_remove, inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1987936 entries, 0 to 1987942
Data columns (total 15 columns):
 #   Column                Dtype
---  ------                -----
 0   Order Via             int8 
 1   Is Completed          bool 
 2   Is Canceled           bool 
 3   Execution Status      int8 
 4   quantity              int64
 5   Price                 int32
 6   Sector Name           int8 
 7   Executed Quantity     int64
 8   Is Closed             bool 
 9   Is Dormant            bool 
 10  Is Profile Suspended  bool 
 11  Is Client Suspended   bool 
 12  Client Type Name      int8 
 13  Expire Year           int8 
 14  Expire Month          int8 
dtypes: bool(6), int32(1), int64(2), int8(6)
memory usage: 75.8 MB
Preprocessed dataset saved to ../data/preprocessed_dataset.csv


In [2]:
df = remove_unnecessary_attributes(df, ["Is Completed","Is Dormant","Is Profile Suspended","Is Client Suspended"])

In [ ]:
# Function to train and evaluate a classifier
def train_and_evaluate_classifier(name, clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred_proba = clf.predict_proba(X_test)[:, 1]  # Probability of positive class
    auc_score = roc_auc_score(y_test, y_pred_proba)
    print(f"\nAUC-ROC Score for {name}: {auc_score}")

# Function to test different models with different dataset features 
def test_models(datasets,classifiers,target):
    i = 0
    for dataset in datasets:
        i = i + 1
        X = dataset.drop(target, axis=1)
        y = dataset[target]

        # Split the dataset into train and test sets
       
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        print("Test case: " + str(i))
        # Train and evaluate classifiers for the first set of data
        for name, clf in classifiers.items():
            train_and_evaluate_classifier(name, clf, X_train, y_train, X_test, y_test)
        print("--------------------------------------------------------------")

datasets = [
    df
]
# Define classifiers
classifiers = {
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=250,n_jobs=4,weights='distance',algorithm='kd_tree'),
    'LogisticRegression': LogisticRegression(random_state=42),
    'XGBoost': xgb.XGBClassifier(objective='binary:logistic', colsample_bytree=0.3, n_estimators=1000, learning_rate=0.05, n_jobs=4, max_depth=5, alpha=10),
    'XGBoost kaggle': xgb.XGBClassifier(objective='binary:logistic', n_estimators=1000, learning_rate=0.05, n_jobs=4),
    'XGBoost chatgpt': xgb.XGBClassifier(objective='binary:logistic', colsample_bytree=0.3, learning_rate=0.1, max_depth=5, alpha=10),
    'Random Forest': RandomForestClassifier(random_state=42),
    'descision Tree': DecisionTreeClassifier()
}

test_models(datasets,classifiers,["Is Closed"])    


Test case: 1


c:\Users\Mohamed\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)



AUC-ROC Score for K-Nearest Neighbors: 0.7627557205089357


c:\Users\Mohamed\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Mohamed\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



AUC-ROC Score for LogisticRegression: 0.6666723388086299

AUC-ROC Score for XGBoost: 0.9207181950021235

AUC-ROC Score for XGBoost kaggle: 0.9475887436789967

AUC-ROC Score for XGBoost chatgpt: 0.8886817490266267


c:\Users\Mohamed\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



AUC-ROC Score for Random Forest: 0.8014937500897577

AUC-ROC Score for descision Tree: 0.6878988461330799
--------------------------------------------------------------
